# W Game

The **W Game** quantum kata is an original series of exercises based on the katas for the GHZ
and CHSH games, introducing an original game inspired by these and referred to here as the "W game".

In it three players (Alice, Bob and Charlie) try to win the following game:

Each of them is given a bit (r, s and t respectively), and
they have to return new bits (a, b and c respectively) so
that the sum (number of true bits) of their output bits corresponds
to the sum of the input bits in the following way:

```
+-------------------------------+---------------------------------------+
|    Number of true bits in     |        Number of true bits in         |
|    input between players      |        output needed to win           |
+-------------------------------+---------------------------------------+
|              0                |               exactly 1               |
|              2                |   0, 2, or 3 (i.e. any number but 1)  |
+-------------------------------+---------------------------------------+
```

The input will have zero or two bits set to true, and one or three bits set to false.
The trick is, the players can not communicate during the game.

**Also,** in this game, all the players have to use the same approach, **if** dependent on the input (i.e. in this game, the team may not have Charlie follow a different protocol from Alice and Bob, **if** any of the protocols depend on the input).  **However,** this restriction only applies to strategies for which the composition of the team's output bits could vary; the rules permit them individual strategies that are independent of the input (such as Bob always outputs true while Alice and Charlie output false) allowing them to ensure that exactly one true bit gets submitted between them.

The W state represents one of the two non-biseparable classes of three-qubit entangled states; the GHZ state represents the other.  These two states cannot be transformed into each other by local (one-qubit) quantum operations, and therefore represent two fundamentally distinct types of tripartite entanglement.  For further information:  https://en.wikipedia.org/wiki/W_state

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl/⌘+Enter.


To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [1]:
%package Microsoft.Quantum.Katas::0.12.20072031

Adding package Microsoft.Quantum.Katas::0.12.20072031: done!

Microsoft.Quantum.Standard::0.12.20072031, Microsoft.Quantum.Katas::0.12.20072031

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Part I. Classical W game


### Task 1.1. Win Condition
**Inputs:** 

  1. Alice, Bob and Charlie's input bits (r, s and t), stored as an array of length 3,

  2. Alice, Bob and Charlie's output bits (a, b and c), stored as an array of length 3.

The input bits will have zero or two bits set to true.

**Output:** 
True if Alice, Bob and Charlie won the W game, that is, if:

* All three input bits were false, and there was exactly one true bit among the team's output bits;
* Two input bits were true, and the number of true bits among the team's output bits was any number other than 1

and false otherwise.

In [2]:
%kata T11_WinCondition_Test 

function WinCondition (rst : Bool[], abc : Bool[]) : Bool {
    mutable abcOnes = 0;
    for (i in abc) {
        if (i) {
            set abcOnes = abcOnes + 1;          // Get number of true bits in output
        }
    }
    if (rst[0] or rst[1] or rst[2]) {
        return (abcOnes != 1);                  // Condition where two input bits are true
    }
    return (abcOnes == 1);                      // Condition where zero input bits are true
}

Success!

### Task 1.2. Random classical strategy

**Input:** The input bit for one of the players (r, s or t).

**Output:** A random bit that this player will output (a, b or c).

If all players use this strategy, they will win about 56% of the time.

In [3]:
%kata T12_RandomClassical_Test 

operation RandomClassicalStrategy (input : Bool) : Bool {
    let coin = Random([0.5, 0.5]);
    return (coin == 1);
}

Success!

### Task 1.3. Simple classical strategy

**Input:** The input bit for one of the players (r, s or t).

**Output:** A bit that this player will output (a, b or c) to achieve a good chance of winning.

All players will use the same strategy.
Multiple naive classical strategies are possible that attain a theoretical win probability of 3/4, or 75% of the time.

In [4]:
%kata T13_SimpleClassical_Test 

operation SimpleClassicalStrategy (input : Bool) : Bool {
    return true;       // One of several simple classical strategies with 75% win odds.
}

Success!

### Task 1.4. Best classical strategy

**Input:** The input bit for one of the players (r, s or t).

**Output:** A bit that this player will output (a, b or c) to maximize their chance of winning.

By rule, all players will use the same strategy.
With this symmetry imposed, the optimal classical strategy should win just over 86% of the time.

Some intermediate probability theory is involved here.

In [12]:
%kata T14_BestClassical_Test 

open Microsoft.Quantum.Math;

// When input is true, always return true;
// when input is false, return true with probability 1/3 and false with probability 2/3.
// This always wins against two true bits.
// Against zero true bits, this maximizes the probability at 4/9 of producing exactly one true output bit to win.
// This leads to a victory chance of (1/4 * 4/9) + (3/4 * 1) = 31/36, or about 86.11%

operation BestClassicalStrategy (input : Bool) : Bool {
    if (input) {
        return true;
    }
    return RandomInt(3) == 0;
}

Success!

### Task 1.5. Referee classical W game

**Inputs:** 

  1. An operation which implements a classical strategy (i.e., takes an input bit and produces an output bit),

  2. An array of 3 input bits that should be passed to the players.

**Output:** 
An array of 3 bits that will be produced if each player uses this strategy.

In [9]:
%kata T15_PlayClassicalW_Test 

operation PlayClassicalW (strategy : (Bool => Bool), inputs : Bool[]) : Bool[] {
    // Input bits and strategy are given, so simply convert them to the output bits.
    let r = inputs[0];
    let s = inputs[1];
    let t = inputs[2];
    let a = strategy(r);
    let b = strategy(s);
    let c = strategy(t);
    return [a, b, c];
}

Success!

## Part II. Quantum W game

In the quantum version of the game, the players still can not
communicate during the game, but they are allowed to share 
qubits from an entangled triple before the start of the game.


### Task 2.1. Entangled triple

**Input:** An array of three qubits in the $|000\rangle$ state.

**Goal:** Create the entangled state $|W\rangle = \frac{1}{\sqrt{3}} \big(|001\rangle + |010\rangle + |100\rangle \big)$ on these qubits.

In [13]:
%kata T21_CreateEntangledTriple_Test 

operation CreateEntangledTriple (qs : Qubit[]) : Unit {
    let theta = ArcSin(1.0 / Sqrt(3.0));
    Ry(2.0 * theta, qs[0]);
    // Starting from |000>, this produces {sqrt(2/3) |000>  +  sqrt(1/3) |100>}.

    (ControlledOnBitString([false], H))([qs[0]], qs[1]);
    // Hadamard-transforms second qubit when first is 0,
    // so this leads to (|000> + |010> + |100>) / sqrt(3).

    (ControlledOnBitString([false, false], X))([qs[0], qs[1]], qs[2]);
    // Applies NOT gate to third qubit when first two are both 0, leading to the W state desired.
}

Success!

### Task 2.2. Quantum strategy

**Inputs:**

  1. The input bit for one of the players (r, s or t),

  2. That player's qubit of the entangled triple shared between the players.

**Goal:** Measure the qubit in the Z basis if the bit is 0 (false), or the X basis if the bit is 1 (true), and return the result.

The state of the qubit after the operation does not matter.

In [14]:
%kata T22_QuantumStrategy_Test 

open Microsoft.Quantum.Measurement;

operation QuantumStrategy (input : Bool, qubit : Qubit) : Bool {
    if (input) {
        let q = MResetX(qubit);
        return (q == One);
    }
    else {
        let q = MResetZ(qubit);
        return (q == One);
    }
}

Success!

### Task 2.3. Play the W game using the quantum strategy

**Input:** Operations that return Alice, Bob and Charlie's output bits (a, b and c) based on
their quantum strategies and given their respective qubits from the entangled triple.
The players have already been told what their starting bits (r, s and t) are.

**Goal:** Return an array of players' output bits (a, b and c).

In [15]:
%kata T23_PlayQuantumW_Test 

operation PlayQuantumW (strategies : (Qubit => Bool)[]) : Bool[] {
    using (qs = Qubit[3]) {
        CreateEntangledTriple(qs);
            
        let a = strategies[0](qs[0]);
        let b = strategies[1](qs[1]);
        let c = strategies[2](qs[2]);

        return [a, b, c];
    }
}

Success!

### Appraisal of quantum strategy

Recall the formula for the win condition:

* The sum of the answer bits must be 1 if the question bits are (0,0,0)
* The sum of the answer bits must be 0, 2, or 3 (i.e. unequal to 1) if the question bits are (0,1,1), (1,0,1) or (1,1,0).

The standard phase vector for a three-qubit system is:

$
\begin{bmatrix}
\psi_{000}\\
\psi_{001}\\
\psi_{010}\\
\psi_{011}\\
\psi_{100}\\
\psi_{101}\\
\psi_{110}\\
\psi_{111}
\end{bmatrix}
$

For the initial W state, this vector has the following values:

$
\begin{bmatrix}
0\\
1/\sqrt{3}\\
1/\sqrt{3}\\
0\\
1/\sqrt{3}\\
0\\
0\\
0
\end{bmatrix}
$

#### Zero input bits true

When the input bits are all false, all three players simply take a Z-basis measurement of their qubit and output the result.  One of the qubits will collapse to true and the others to false, so the win condition is met.

#### Two input bits true

When two input bits are true, the players holding the true bits will measure their qubit in the X basis, while the third measures in the Z.  This operation is equivalent to the two players holding true input bits performing a Hadamard transform on their qubit and then measuring in the Z.  Consider the set of input bits (0,1,1), for which Bob and Charlie perform a Hadamard transform on their qubits.  The relevant $8 \times 8$ operator matrix is:

$
I \otimes H \otimes H =
\begin{bmatrix}
1/2 & 1/2 & 1/2 & 1/2 & 0 & 0 & 0 & 0\\
1/2 & -1/2 & 1/2 & -1/2 & 0 & 0 & 0 & 0\\
1/2 & 1/2 & -1/2 & -1/2 & 0 & 0 & 0 & 0\\
1/2 & -1/2 & -1/2 & 1/2 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 1/2 & 1/2 & 1/2 & 1/2\\
0 & 0 & 0 & 0 & 1/2 & -1/2 & 1/2 & -1/2\\
0 & 0 & 0 & 0 & 1/2 & 1/2 & -1/2 & -1/2\\
0 & 0 & 0 & 0 & 1/2 & -1/2 & -1/2 & 1/2
\end{bmatrix}
$

Operating on the W state with this matrix leads to:

$
\begin{bmatrix}
1/\sqrt{3}\\
0\\
0\\
-1/\sqrt{3}\\
1/\sqrt{12}\\
1/\sqrt{12}\\
1/\sqrt{12}\\
1/\sqrt{12}
\end{bmatrix}
$

The players measure this entangled state in the Z basis, with the following outcomes possible:

```
+-----------------------+--------------------------------+
|   No. of |1> Qubits   |   Probability of Observation   |
+-----------------------+--------------------------------+
|         0             |              1/3               |
|         1             |              1/12              |
|         2             |              1/2               |
|         3             |              1/12              |
+-----------------------+--------------------------------+
```

The team only loses in the event of one qubit measured at $|1\rangle$ and the others at $|0\rangle$.  This occurs $1/12$ of the time, so the team's total winning probability is $1/4 + (3/4)(11/12) = 1/4 + 11/16 = 15/16$, or a 93.75% chance.  This is a substantial improvement on the 86.11% chance offered by the optimum symmetrical classical strategy in Task 1.4 above.